In [31]:
import tensorflow as tf
import datetime
import os
import pandas as pd
import numpy as np
#import witwidget

from witwidget.notebook.visualization import WitConfigBuilder, WitWidget

log_dir = os.path.join(
    "logs",
    "fit",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
  examples = []
  if columns == None:
    columns = df.columns.values.tolist()
  for index, row in df.iterrows():
    example = tf.train.Example()
    for col in columns:
      if df[col].dtype is np.dtype(np.int64):
        example.features.feature[col].int64_list.value.append(int(row[col]))
      elif df[col].dtype is np.dtype(np.float64):
        example.features.feature[col].float_list.value.append(row[col])
      elif row[col] == row[col]:
        example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
    examples.append(example)
  return examples

num_datapoints = 1000
tool_height_in_px = 700
df = pd.read_csv(csv_path)

test_examples = df_to_examples(df[0:num_datapoints])

lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(20, activation="tanh",unit_forget_bias=True, dropout=0.0, recurrent_dropout=0.0,return_sequences=True),
    tf.keras.layers.Dense(units=1)
])


In [32]:
feature_spec = df.iloc[:,0:12].values

In [33]:
config_builder = WitConfigBuilder(test_examples[0:num_datapoints]).set_estimator_and_feature_spec(lstm_model, feature_spec).set_model_type('classification')

In [34]:
WitWidget(config_builder, height=tool_height_in_px)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…